In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np

In [ ]:
# Import data from "tsp.text"
url_data = (
    "https://raw.githubusercontent.com/VinceZ714/MECS-4510-Evolutionary-Computation/main/tsp.txt"
)

data = pd.read_csv(url_data)
pointx = data.x
pointy = data.y


In [ ]:
def route_distance(path: list, pointlist: list):
  distance = 0
  for i in range(0, len(path) - 1):
    frompoint = pointlist[path[i]]
    topoint = pointlist[path[i+1]]
    x0 = frompoint[0]
    y0 = frompoint[1]
    x1 = topoint[0]
    y1 = topoint[1]
    distance += (((x1-x0)**2)+((y1-y0)**2))**0.5
  return distance

In [ ]:
def adjacent_test(point1: list, point2: list):
  dx = abs(point1[0] - point2[0])
  dy = abs(point1[1] - point2[1])
  if dx < 0.005 and dy < 0.005:
    return 1
  else:
    return 0

In [ ]:
def point_swap (path: list, p1:list, p2:list):
  path[p1], path[p2] = path[p2], path[p1]
  return path

In [ ]:
def pop_generate(pop_size: list, pointlist: list):
  org_path = []
  path = []
  distance = []
  for i in range(len(pointlist)):
    org_path.append(i)
  for i in range(pop_size):
    temp_path = random.sample(org_path, len(pointlist))
    path.append(temp_path)
    temp_distance = route_distance(temp_path, pointlist)
    distance.append(temp_distance)
  population = list(zip(distance,path))
  return population

In [ ]:
def selection(population: list, selectsize: int):
  select_pop = []
  population = sorted(population, key = lambda x: x[0], reverse = False)
  for i in range(selectsize):
    select_pop.append(population[i])
  return select_pop

In [ ]:
def breed(parent1: list, parent2: list):
  child = []
  cp1 = []
  cp2 = []
  cutpoint1 = random.randrange(len(parent1))
  cutpoint2 = random.randrange(len(parent1))
  genestart = min(cutpoint1, cutpoint2)
  geneend = max(cutpoint1, cutpoint2)
  for i in range(genestart, geneend):
    cp1.append(parent1[i])
  cp2 = [point for point in parent2 if point not in cp1]
  child = cp1 + cp2
  return child

In [ ]:
def pop_children_born(population: list, popsize: int):
  path = []
  distance = []
  for i in range(len(population)):
    path.append(population[i][1])
    distance.append(population[i][0])
  for i in range(len(population)):
    parent1 = current_pop[i][1]
    parent2 = current_pop[random.randrange(len(current_pop))][1]
    child = breed(parent1, parent2)
    child = mutate(child, pointlist)
    distance.append(route_distance(child, pointlist))
    path.append(child)
  temp_pop = list(zip(distance,path))
  return temp_pop

In [ ]:
def mutate(child: list, pointlist: list):
  point1 = random.randrange(len(child))
  point2 = random.randrange(len(child))
  a = adjacent_test(pointlist[child[point1]], pointlist[child[point2]])
  if a == 1:
    point2 = random.randrange(len(child))
  child[point1], child[point2] = child[point2], child[point1]
  return child

Random Search Method

In [ ]:
# Variable set up
pointlist = []
org_path = [] 
evaluation = 300000
count = 0
fitness_curve_rs = []
distance_curve_rs = []
evaluation_num_rs =[]
best_path_rs = None
best_distance_rs = None
for i in range(len(pointx)):
  x = pointx[i]
  y = pointy[i]
  pointlist.append([x, y])
for i in range(len(pointlist)):
  org_path.append(i)
for i in range(evaluation):
  evaluation_num_rs.append(i)

In [ ]:
# Create random path and find the shortest distance
for i in range(evaluation):
  temp_path = random.sample(org_path, len(pointlist))
  temp_distance = route_distance(temp_path, pointlist)
  if best_path_rs == None:
    best_path_rs = temp_path
    best_distance_rs = temp_distance
  else:
    if temp_distance < best_distance_rs:
      best_path_rs = temp_path
      best_distance_rs = temp_distance
  distance_curve_rs.append(best_distance_rs)
  fitness_curve_rs.append(1/best_distance_rs)

Hill Climber Method

In [ ]:
# Variable set up
pointlist = []
org_path = [] 
evaluation = 300000
evaluation_num_hc = []
fitness_curve_hc = []
distance_curve_hc = []
best_path_hc = None
best_distance_hc = None
for i in range(len(pointx)):
  x = pointx[i]
  y = pointy[i]
  pointlist.append([x, y])
for i in range(len(pointlist)):
  org_path.append(i)
for i in range(evaluation):
  evaluation_num_hc.append(i)

In [ ]:
# Create random path first
# Change the adjacent points to find the shortest distance
for i in range(0,evaluation):
  if i == 0:
    org_path = random.sample(org_path, len(pointlist))
    temp_distance = route_distance(temp_path, pointlist)
    best_path_hc = temp_path
    best_distance_hc = temp_distance
  else:
    point1 = random.randrange(len(temp_path))
    point2 = random.randrange(len(temp_path))
    a = adjacent_test(pointlist[temp_path[point1]],pointlist[temp_path[point2]])
    if a == 1:
      point2 = random.randrange(len(temp_path))
    temp_path[point1], temp_path[point2] = temp_path[point2], temp_path[point1]
    temp_distance = route_distance(temp_path, pointlist)
    if temp_distance < best_distance_hc:
      best_path_hc = temp_path
      best_distance_hc = temp_distance
    else:
      temp_path[point1], temp_path[point2] = temp_path[point2], temp_path[point1]
  distance_curve_hc.append(best_distance_hc)
  fitness_curve_hc.append(1/best_distance_hc)
  print(i)
  print(best_distance_hc)

Generation Algorithm

In [ ]:
# Variable set up
pointlist = []
org_path = [] 
evaluation = 300000
fitness_curve_ga = []
distance_curve_ga = []
evaluation_num_ga =[]
best_path_ga = None
best_distance_ga = None
popsize = 20
selectsize = int(popsize/2)
for i in range(len(pointx)):
  x = pointx[i]
  y = pointy[i]
  pointlist.append([x, y])
for i in range(len(pointlist)):
  org_path.append(i)
for i in range(evaluation):
  evaluation_num_ga.append(i)

In [ ]:
for i in range(evaluation):
  if i == 0: 
    initial_pop = pop_generate(popsize, pointlist)
    select_pop = selection(initial_pop, selectsize)
    best_path_ga = select_pop[0][1]
    best_distance_ga = select_pop[0][0]
    current_pop = select_pop
  else:
    current_pop = pop_children_born(current_pop, popsize)
    select_pop = selection(current_pop, selectsize)
    if select_pop[0][0] < best_distance_ga:
      best_path_ga = select_pop[0][1]
      best_distance_ga = select_pop[0][0]
      print(i)
      print(best_distance_ga)
    current_pop = select_pop
  distance_curve_ga.append(best_distance_ga)
  fitness_curve_ga.append(1/best_distance_ga)

Plot

In [ ]:
url_rss = (
    "/content/sample_data/Error_rs Short.csv"
)
url_hcs = (
    "/content/sample_data/Error_hc Short.csv"
)
url_gas = (
    "/content/sample_data/Error_ga Short.csv"
)
url_rsl = (
    "/content/sample_data/Error_rs Long.csv"
)
url_hcl = (
    "/content/sample_data/Error_hc Long.csv"
)
url_gal = (
    "/content/sample_data/Error_ga Long.csv"
)
url_harshs = (
    "/content/sample_data/Error_ga_harsh Short.csv"
)
url_harshl = (
    "/content/sample_data/Error_ga_harsh Long.csv"
)

rss = pd.read_csv(url_rss)
hcs = pd.read_csv(url_hcs)
gas = pd.read_csv(url_gas)
rsl = pd.read_csv(url_rsl)
hcl = pd.read_csv(url_hcl)
gal = pd.read_csv(url_gal)
harshs = pd.read_csv(url_harshs)
harshl = pd.read_csv(url_harshl)

def plot_cal(pandalist: pd.core.frame.DataFrame):
  average = []
  for i in range(len(pandalist)):
    average.append(pandalist.loc[i].mean())
  error = np.std(average)/np.sqrt(len(pandalist.loc[0]))
  return average, error

average_distance_rss, error_distance_rss = plot_cal(rss) 
average_distance_hcs, error_distance_hcs = plot_cal(hcs) 
average_distance_gas, error_distance_gas = plot_cal(gas)

average_fitness_rss, error_fitness_rss = plot_cal(1/rss) 
average_fitness_hcs, error_fitness_hcs = plot_cal(1/hcs)
average_fitness_gas, error_fitness_gas = plot_cal(1/gas)

average_distance_rsl, error_distance_rsl = plot_cal(rsl) 
average_distance_hcl, error_distance_hcl = plot_cal(hcl) 
average_distance_gal, error_distance_gal = plot_cal(gal)

average_fitness_rsl, error_fitness_rsl = plot_cal(1/rsl) 
average_fitness_hcl, error_fitness_hcl = plot_cal(1/hcl)
average_fitness_gal, error_fitness_gal = plot_cal(1/gal)

average_distance_harshs, error_distance_harshs = plot_cal(harshs)

average_fitness_harshs, error_fitness_harshs = plot_cal(1/harshs)

average_distance_harshl, error_distance_harshl = plot_cal(harshl)

average_fitness_harshl, error_fitness_harshl = plot_cal(1/harshl)

In [ ]:
evaluation_num = []
for i in range(evaluation):
  evaluation_num.append(i)

In [ ]:
# Shortest Path
plt.figure(1)

plt.errorbar(evaluation_num, average_distance_rss, yerr = error_distance_rss, 
             errorevery = 50000, capsize = 3,
             label='Random Search')


plt.errorbar(evaluation_num, average_distance_hcs, yerr = error_distance_hcs, 
             errorevery = 50000, capsize = 3,
             label = 'Hill Climber')

plt.errorbar(evaluation_num, average_distance_gas, 
             yerr = error_distance_gas, 
             errorevery = 50000, capsize = 3,
             label = 'Generation Algorithm')

plt.errorbar(evaluation_num, average_distance_harshs, 
             yerr = error_distance_harshs, 
             errorevery = 50000, capsize = 3,
             label = 'GA Harsh Selection')

plt.xlim(0,evaluation)
plt.ylabel('Path Distance')
plt.xlabel('Evolution')
plt.title('Distance (Shortest Path)')
plt.legend()


plt.figure(2)

plt.errorbar(evaluation_num, average_fitness_rss, yerr = error_fitness_rss,            
             errorevery = 50000, capsize = 3,
             label='Random Search')

plt.errorbar(evaluation_num, average_fitness_hcs, yerr = error_fitness_hcs,
             errorevery = 50000, capsize = 3,
             label = 'Hill Climber')

plt.errorbar(evaluation_num, average_fitness_gas, 
             yerr = error_fitness_gas, 
             errorevery = 50000, capsize = 3,
             label = 'Generation Algorithm')

plt.errorbar(evaluation_num, average_fitness_harshs, 
             yerr = error_fitness_harshs, 
             errorevery = 50000, capsize = 3,
             label = 'GA Harsh Selection')

plt.xlim(0,evaluation)
plt.ylabel('Fitness')
plt.xlabel('Evolution')
plt.title('Learning Curve(Shortest Path)')
plt.legend()


In [ ]:
# Longest Path
plt.figure(3)

plt.errorbar(evaluation_num, average_distance_rsl, yerr = error_distance_rsl, 
             errorevery = 50000, capsize = 3,
             label='Random Search')


plt.errorbar(evaluation_num, average_distance_hcl, yerr = error_distance_hcl, 
             errorevery = 50000, capsize = 3,
             label = 'Hill Climber')

plt.errorbar(evaluation_num, average_distance_gal, 
             yerr = error_distance_gal, 
             errorevery = 50000, capsize = 3,
             label = 'Generation Algorithm')

plt.errorbar(evaluation_num, average_distance_harshl, 
             yerr = error_distance_harshl, 
             errorevery = 50000, capsize = 3,
             label = 'GA Harsh Selection')

plt.xlim(0,evaluation)
plt.ylabel('Path Distance')
plt.xlabel('Evolution')
plt.title('Distance (Longest Path)')
plt.legend()


plt.figure(4)

plt.errorbar(evaluation_num, average_fitness_rsl, yerr = error_fitness_rsl,            
             errorevery = 50000, capsize = 3,
             label='Random Search')

plt.errorbar(evaluation_num, average_fitness_hcl, yerr = error_fitness_hcl,
             errorevery = 50000, capsize = 3,
             label = 'Hill Climber')

plt.errorbar(evaluation_num, average_fitness_gal, 
             yerr = error_fitness_gal, 
             errorevery = 50000, capsize = 3,
             label = 'Generation Algorithm')

plt.errorbar(evaluation_num, average_fitness_harshl, 
             yerr = error_fitness_harshl, 
             errorevery = 50000, capsize = 3,
             label = 'GA Harsh Selection')

plt.xlim(0,evaluation)
plt.ylabel('Fitness')
plt.xlabel('Evolution')
plt.title('Learning Curve (Longest Path)')
plt.legend()

In [ ]:
plt.figure(5)
plt_pathx_ga = []
plt_pathy_ga = []
for i in range(0, len(pointlist) - 1):
  plt_pathx_ga.append(data.x[best_path_ga[i]])
  plt_pathy_ga.append(data.y[best_path_ga[i]])
plt.plot(plt_pathx_ga, plt_pathy_ga)
plt.ylabel('y')
plt.xlabel('x')
plt.title('Shortest Path')